In [11]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests
import logging

logic: 
1. first get the max number of external slots: 200
2. then filter all the validators who are eligible to be selected next epoch; if the total number of validators is lower than 200, then every filtered validator should be elected next epoch; if total number is higher than 200, we need to first sort these validators based on their total delegations, and get the top 200 validators.
3. wait for one epoch, to check whether those filtered validators are elected

In [26]:
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)


In [27]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [28]:
def getCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [29]:
def getAllValidator():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result'] 

In [30]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [31]:
def getEpoch():
    method = "hmy_getEpoch"
    params = []
    epoch = get_information(method, params)['result']
    return int(epoch, 16)

In [32]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [40]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [41]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [33]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result']

In [34]:
def getEligibleValidator():
    eligible = []
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
            address = i['validator']['address']
            eligible.append(address)
    return eligible

In [43]:
def E1_test():
    global curr_test
    logging.info(f"Test-E1: A staked validator whose stake is in the top #slots stakes is always considered for election")
    try:
        committees = getCommittees()
        slot = committees['external-slot-count']
        block, last_block = getCurrentAndLastBlock()
        if block == last_block:
            logger.info(f"current block is the last block in epoch, waiting for the new epoch...")
            new_block = block+1
            while block < new_block:
                block = getBlockNumber()
            block, last_block = getCurrentAndLastBlock()
        logger.info(f"current block, {block}, begin collecting eligible validators...")
        # get top #slots nodes who are eligible to elected next epoch
        validator_infos = getAllValidatorInformation()
        eligible = []
        stake = dict()
        for i in validator_infos:
            if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
                address = i['validator']['address']
                eligible.append(address)
                stake[address] = i['total-delegation']

        if len(eligible) > slot:
            sorted_stake = sorted(stake.items(), key=lambda kv: kv[1], reverse = True)
            eligible = [kv[0] for kv in sorted_stake[:slot]]

        # wait for epoch changes
        while block < last_block+1:
            block = getBlockNumber()
        logger.info(f"first block in new epoch reached, {block}, will wait for 5 seconds to begin testing...")
        time.sleep(5)
        # check whether the eligible validators are selected
        validator_infos = getAllValidatorInformation()
        flag = True
        for i in validator_infos:
            if i in eligible:  
                if i['epos-status'] != 'currently elected':
                    logger.warning(f"Test E1: Fail")
                    logger.warning(f"validator {i['validator']['address']} who is eligible to be elected is not elected")
                    flag = False
    except TypeError as e:
        logger.error(f"error: {e}")
    if flag:
        logger.info(f"Test E1: Succeed")
        return True
    else:
        return False

In [44]:
E1_test()

2020-04-11 00:46:38,953:Test-E1: A staked validator whose stake is in the top #slots stakes is always considered for election
2020-04-11 00:46:39,556:current block, 8382, begin collecting eligible validators...


current and last block numbers 8382 8397


2020-04-11 00:48:40,668:first block in new epoch reached, 8398, will wait for 5 seconds to begin testing...
2020-04-11 00:48:46,112:Test E1: Succeed


True

the info of the exceptions: 

In [9]:
i = 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy'
info = getValidatorInfo(i)
info

{'validator': {'bls-public-keys': ['2eba33b81d0aacb22e793625077d0c57dd6d38d753bbf39dc720dea0f4a5c056c81d8967a99c2cf1811d0327b049ab16',
   '394d726edbb908fa71bdb2b417dd0fe7c77d9f86b3a0b1e1b51d11e26dd1cddb4e21269c3e1d08e75d43ec98ff4ca582',
   '73ac6379c785142d46bf8423469639fa1dfb9e416e97206cff2c07522a18999a54faaf9cbcad1e34a6c2dc770224848e',
   'b682337f407ec929e7cc18e5103da8a038d63118990b9d8124b31836d8b394a695f16d88b6a32b2543b1dcaaba99a582'],
  'last-epoch-in-committee': 0,
  'min-self-delegation': 322222000000000000000000,
  'max-total-delegation': 10000000000000000000000000,
  'rate': '0.030000000000000000',
  'max-rate': '0.700000000000000000',
  'max-change-rate': '0.030000000000000000',
  'update-height': 1014,
  'name': 'leo-s2',
  'identity': 'leo-sentry-s2',
  'website': 'https://sentry2.leochen.net',
  'security-contact': 'sentrys2@leochen.net',
  'details': 'sentry node of leo chen, shard2',
  'creation-height': 1014,
  'address': 'one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy',
 

In [10]:
i = 'one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04'
info = getValidatorInfo(i)
info

{'validator': {'bls-public-keys': ['4db5eeb4ff5379f7ba51d522fb17a07571cf427fae86cab31e9927e5048339d7a3c40941b1ce485f75f750aabbf69d82'],
  'last-epoch-in-committee': 0,
  'min-self-delegation': 10000000000000000000000,
  'max-total-delegation': 100000000000000000000000,
  'rate': '0.100000000000000000',
  'max-rate': '0.100000000000000000',
  'max-change-rate': '0.100000000000000000',
  'update-height': 25363,
  'name': 'Yishuang-2',
  'identity': 'Yishuang sentry',
  'website': 'yishuang@harmony.one',
  'security-contact': 'shuang',
  'details': 'Yishuang’s second validator',
  'creation-height': 25363,
  'address': 'one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04',
  'delegations': [{'delegator-address': 'one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04',
    'amount': 10000000000000000000000,
    'reward': 0,
    'undelegations': []}]},
 'current-epoch-performance': None,
 'metrics': None,
 'total-delegation': 10000000000000000000000,
 'currently-in-committee': False,
 'epos-status': 'eligible 